In [1]:
import os

print("Project Directory:", os.getcwd())
print("Known Faces:", os.listdir("known_faces"))
print("Unknown Faces:", os.listdir("unknown_faces"))


Project Directory: C:\Users\anand\face_recognition_project
Known Faces: []
Unknown Faces: []


In [13]:
import face_recognition
import cv2
import os
import numpy as np
from PIL import Image

# 📁 Step 1: Load known faces
known_face_encodings = []
known_face_names = []

known_dir = "known_faces"

for filename in os.listdir(known_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(known_dir, filename)
        image = face_recognition.load_image_file(image_path)
        encodings = face_recognition.face_encodings(image)

        if encodings:
            known_face_encodings.append(encodings[0])
            known_face_names.append(os.path.splitext(filename)[0])
        else:
            print(f"⚠️ No face found in {filename}")

print(f"✅ Loaded {len(known_face_names)} known faces.\n")

# 📸 Step 2: Recognize faces in unknown images
unknown_dir = "unknown_faces"

for filename in os.listdir(unknown_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(unknown_dir, filename)
        image = face_recognition.load_image_file(image_path)

        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations)

        pil_image = Image.fromarray(image)
        draw = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)

            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # 🟩 Draw rectangle and label
            cv2.rectangle(draw, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(draw, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # 🖼️ Display the image with labels
        print(f"\n📷 Recognitions in {filename}:")
        Image.fromarray(cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)).show()


✅ Loaded 3 known faces.


📷 Recognitions in boy.png:

📷 Recognitions in guess_me.jpeg:

📷 Recognitions in IMG_20230830_143157580.jpg:

📷 Recognitions in Screenshot 2025-07-08 193256.png:

📷 Recognitions in Screenshot 2025-07-08 193614.png:


In [3]:
import face_recognition
import cv2
import os
import numpy as np

# Load known faces
known_face_encodings = []
known_face_names = []

known_dir = "known_faces"

for filename in os.listdir(known_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(known_dir, filename)
        image = face_recognition.load_image_file(image_path)
        encodings = face_recognition.face_encodings(image)

        if encodings:
            known_face_encodings.append(encodings[0])
            known_face_names.append(os.path.splitext(filename)[0])
        else:
            print(f"⚠️ No face found in {filename}")

print(f"✅ Loaded {len(known_face_names)} known faces.")

# Start webcam
video_capture = cv2.VideoCapture(0)

print("🎥 Press 'q' to quit")

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Resize frame for faster processing (optional)
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Detect faces
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    # Match each face
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        if len(face_distances) > 0:
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

        # Scale face locations back to original size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw box and label
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Show result
    cv2.imshow('Webcam Face Recognition', frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release webcam
video_capture.release()
cv2.destroyAllWindows()


✅ Loaded 3 known faces.
🎥 Press 'q' to quit
